In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 cURL 入門 Vertex AI Gemini API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/gemini/getting-started/intro_gemini_curl.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 上執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fdoggy8088%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_curl.zh.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 上執行
    </a>
  </td>       
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/gemini/getting-started/intro_gemini_curl.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/main/gemini/getting-started/intro_gemini_curl.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Eric Dong](https://github.com/gericdong), [Polong Lin](https://github.com/polong-lin) |


## 概述

### Gemini
Gemini 是一個由 Google Deepmind 開發的生成式 AI 模型系列。Gemini 模型支援輸入中包含文字、影像和影片的提示，並支援文字回應作為輸出。

### Vertex AI Gemini API

Vertex AI Gemini API 提供一個統一的介面，用於與 Gemini 模型互動。在 Gemini API 中有兩個 Gemini 1.0 Pro 模型可用：
* **Gemini 1.0 Pro 模型** (`gemini-1.0-pro`)：經過微調以處理自然語言任務，例如分類、摘要、萃取和撰寫。
* **Gemini 1.0 Pro Vision 模型** (`gemini-1.0-pro-vision`)：多模態模型，支援在文字或聊天提示中加入影像和影片，以取得文字回應。

你可以使用以下方法與 Gemini API 互動：

* 使用 [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) 快速測試和產生指令。
* 在 Cloud Shell 中使用 cURL 指令。
* 在 Jupyter notebook 中使用 Python 的 Vertex AI SDK

本筆記本重點說明如何使用 **cURL 指令** 呼叫 Vertex AI Gemini API。

如需更多資訊，請參閱 [Vertex AI 上的生成式 AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) 文件。


### 目標

在本教學課程中，你將學習如何使用 cURL 命令搭配 Vertex AI Gemini API 來與 Gemini 1.0 Pro (`gemini-1.0-pro`) 模型和 Gemini 1.0 Pro Vision (`gemini-1.0-pro-vision`) 模型進行互動。

你將完成下列任務：

- 安裝 Python SDK。
- 使用 Vertex AI Gemini API 與每個模型進行互動。
  - Gemini 1.0 Pro (`gemini-1.0-pro`) 模型：
    - 從文字提示中產生文字。
    - 探索各種功能與設定選項。
  - Gemini 1.0 Pro Vision (`gemini-1.0-pro-vision`) 模型：
    - 從影像和文字提示產生文字。
    - 從影片產生文字。


### 成本
本教學課程使用 Google Cloud 的計費元件：

- Vertex AI

深入了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)並使用 [定價計算器](https://cloud.google.com/products/calculator/)，根據你的預計使用量產生成本估算。


## 開始使用


### 驗證筆記本環境 (僅限於 Colab) 

如果你於 Google Colab 上執行此筆記本，請執行下列單元格來驗證你的環境。

如果你使用 [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)，則不需要執行此步驟。


In [1]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 設定 Google Cloud 專案


In [2]:
PROJECT_ID = "[your-project_id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 定義 cURL 指令的環境變數

這些環境變數用於建構 cURL 指令。


In [3]:
import os

os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["LOCATION"] = LOCATION
os.environ["API_ENDPOINT"] = f"{LOCATION}-aiplatform.googleapis.com"

## 使用 Gemini 1.0 Pro 模型

Gemini 1.0 Pro (`gemini-1.0-pro`) 模型專門用於自然語言任務，例如分類、摘要、提取和寫作。

### 從文字產生文字

將文字提示傳送給模型。Gemini 1.0 Pro (`gemini-1.0-pro`) 模型提供串流回應機制。有了這種方法，你不必等到完整回應；你可以一有可用片段就開始處理。


In [4]:
%%bash

MODEL_ID="gemini-1.0-pro"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Why is the sky blue?" }
    }
  }'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "* The sky is blue because of the way sunlight interacts with molecules in the Earth's atmosphere.\n* When sunlight enters the atmosphere, it is scattered in"
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.04288224,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.025323061
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.15253653,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.060197048
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.107063465,
          "severity": "HARM_SEV

100  5743    0  5642  100   101   2078     37  0:00:02  0:00:02 --:--:--  2114


          "severityScore": 0.026006784
        }
      ]
    }
  ]
}
,
{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": " more of the atmosphere to reach our eyes.\n* This means that more of the blue light is scattered away, and we see more of the other colors of light, such as red and orange."
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.058560826,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.039787807
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.058776557,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.053799648
        },
        {
          "category": "HARM_CATEGORY_HARASSM

### 模型參數

你傳送至模型的每個提示詞都包含參數值，用以控制模型如何生成回應。模型可以針對不同的參數值產生不同的結果。你可以使用不同的模型參數來實驗，以查看結果如何改變。


In [5]:
%%bash

MODEL_ID="gemini-1.0-pro-vision"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {"text": "Describe this image"},
        {"file_data": {
          "mime_type": "image/png",
          "file_uri": "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg"
        }}
      ]
    },
    "generation_config": {
      "temperature": 0.2,
      "top_p": 0.1,
      "top_k": 16,
      "max_output_tokens": 2048,
      "candidate_count": 1,
      "stop_sequences": []
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3315    0  2712  100   603    936    208  0:00:02  0:00:02 --:--:--  1143


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": " This is a photo of a cat walking in the snow. The cat is brown and white, and it has a long tail. The cat is walking towards"
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.031618766,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.06669065
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.061424047,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.0575992
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.066085376,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "se

### 聊天

Gemini 1.0 Pro 模型支援自然多輪對話，非常適合需要來回互動的文字任務。

僅當內容表示對話中的輪次時才指定 `role` 欄位。你可以將 `role` 設定為下列值之一：`user`、`model`。


In [6]:
%%bash

MODEL_ID="gemini-1.0-pro"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d '{
    "contents": [
      {
        "role": "user",
        "parts": [
          { "text": "Hello" }
        ]
      },
      {
        "role": "model",
        "parts": [
          { "text": "Hello! I am glad you could both make it." }
        ]
      },
      {
        "role": "user",
        "parts": [
          { "text": "So what is the first order of business?" }
        ]
      }
    ]
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3167    0  2769  100   398   2920    419 --:--:-- --:--:-- --:--:--  3337


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": "The first order of business is to introduce ourselves. I am Generative Pre-trained Transformer 3, or GPT-3 for short. I am a large"
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.034036152,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.036838707
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.042722195,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.09138211
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.08479541,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
       

### 函式呼叫

函式呼叫可讓你在程式碼中建立函式描述，然後在請求中將該描述傳遞給語言模型。此範例是傳遞能傳回電影播放地點資訊之函式描述的範例。此請求包含多個函式宣告，例如 `find_movies` 與 `find_theaters`。

進一步了解 [函式呼叫](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling)。


In [7]:
%%bash

MODEL_ID="gemini-1.0-pro"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1beta1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d '{
  "contents": {
    "role": "user",
    "parts": {
      "text": "Which theaters in Mountain View show Barbie movie?"
    }
  },
  "tools": [
    {
      "function_declarations": [
        {
          "name": "find_movies",
          "description": "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "description": {
                "type": "string",
                "description": "Any kind of description including category or genre, title words, attributes, etc."
              }
            },
            "required": [
              "description"
            ]
          }
        },
        {
          "name": "find_theaters",
          "description": "find theaters based on location and optionally movie title which are is currently playing in theaters",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              }
            },
            "required": [
              "location"
            ]
          }
        },
        {
          "name": "get_showtimes",
          "description": "Find the start times for movies playing in a specific theater",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              },
              "theater": {
                "type": "string",
                "description": "Name of the theater"
              },
              "date": {
                "type": "string",
                "description": "Date for requested showtime"
              }
            },
            "required": [
              "location",
              "movie",
              "theater",
              "date"
            ]
          }
        }
      ]
    }
  ]
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4073    0  1506  100  2567   2192   3736 --:--:-- --:--:-- --:--:--  5928


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "functionCall": {
              "name": "find_theaters",
              "args": {
                "location": "Mountain View, CA",
                "movie": "Barbie"
              }
            }
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.08555634,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.116965584
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.086169556,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.14390807
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probab

## 使用 Gemini 1.0 Pro 視覺模型

Gemini 1.0 Pro 視覺 (`gemini-1.0-pro-vision`) 是一個多模態模型，支援在文字或對話提示中加入影像和影片，以產生文字回應。

**注意：** Gemini 1.0 Pro 視覺模型不支援純文字提示。請改用 Gemini 1.0 Pro 模型來處理純文字提示。


### 從 Google Cloud Storage 下載圖片


In [8]:
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

Copying gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg...
/ [1 files][ 17.4 KiB/ 17.4 KiB]                                                
Operation completed over 1 objects/17.4 KiB.                                     


### 從區域圖片產生文字

指定要內嵌在提示中的圖片或影片的 [base64](https://zh.wikipedia.org/wiki/Base64) 編碼以及 `mime_type` 欄位。圖片支援的 [媒體類型](https://zh.wikipedia.org/wiki/%E5%AA%92%E4%BD%93%E7%B1%BB%E9%80%81) 包括 `image/png` 和 `image/jpeg`。


In [9]:
%%bash

# Encode image data in base64
# NOTE: This command only works on Linux.
data=$(base64 -w 0 image.jpg)

MODEL_ID="gemini-1.0-pro-vision"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d "{
      'contents': {
        'role': 'USER',
        'parts': [
          {
            'text': 'Is it a cat?'
          },
          {
            'inline_data': {
              'data': '${data}',
              'mime_type':'image/jpeg'
            }
          }
        ]
       }
     }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25431    0  1337  100 24094    522   9419  0:00:02  0:00:02 --:--:--  9941


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": " Yes, it is a cat."
          }
        ]
      },
      "finishReason": "STOP",
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.07107367,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.11143445
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.14487323,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.12389466
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.15482272,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.15115306
        },
        {
          "category": "HARM_CATEG

### 從 Google Cloud Storage 上的圖片產生文字

指定要包含在提示中的圖片之雲端儲存 URI。儲存檔案的儲存空間必須與發出請求的同一個 Google Cloud 專案。你也必須指定 `mime_type` 欄位。受支援的圖片 MIME 類型包括 `image/png` 和 `image/jpeg`。


In [10]:
%%bash

MODEL_ID="gemini-1.0-pro-vision"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {
          "text": "Describe this image"
        },
        {
          "file_data": {
            "mime_type": "image/png",
            "file_uri": "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg"
          }
        }
      ]
    },
    "generation_config": {
      "temperature": 0.2,
      "top_p": 0.1,
      "top_k": 16,
      "max_output_tokens": 2048,
      "candidate_count": 1,
      "stop_sequences": []
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3361    0  2712  100   649    871    208  0:00:03  0:00:03 --:--:--  1080


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": " This is a photo of a cat walking in the snow. The cat is brown and white, and it has a long tail. The cat is walking towards"
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.031618766,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.06669065
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.061424047,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.0575992
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.066085376,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "se

### 根據影片檔案產生文字內容

請指定雲端儲存服務中影片的 URI，以納入提示中。儲存檔案的儲存區必須與傳送請求的同一 Google Cloud 專案相同。你還必須指定 `mime_type` 欄位。影片支援的 MIME 類型包括 `video/mp4`。


In [11]:
%%bash

MODEL_ID="gemini-1.0-pro-vision"

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}/v1/projects/${PROJECT_ID}/locations/${LOCATION}/publishers/google/models/${MODEL_ID}:streamGenerateContent \
  -d \
'{
    "contents": {
      "role": "USER",
      "parts": [
        {
          "text": "Answer the following questions using the video only. What is the profession of the main person? What are the main features of the phone highlighted?Which city was this recorded in?Provide the answer JSON."
        },
        {
          "file_data": {
            "mime_type": "video/mp4",
            "file_uri": "gs://github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4"
          }
        }
      ]
    }
  }'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3248    0  2734  100   514    138     26  0:00:19  0:00:19 --:--:--   604


[{
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "text": " ```json\n{\n  \"profession\": \"photographer\",\n  \"features\": \"Night Sight, Video Boost\",\n  \"city\": \"Tokyo"
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.048948623,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.07463504
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.23178805,
          "severity": "HARM_SEVERITY_LOW",
          "severityScore": 0.20370755
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE",
          "probabilityScore": 0.06828854,
          "severity": "HARM_SEVERITY_NEGLIGIBLE",
          "severityScore": 0.0